In [29]:
from code_analysis import CFG 
from code_analysis import CFGReader
from code_analysis import AST
from code_analysis import ASTReader
import json

cfg = CFGReader().read_cfg('/mnt/c/Users/Othman/Desktop/TP-LOG6302/log6302a_lab5/part_1/file_2.php.cfg.json')
ast = ASTReader().read_ast('/mnt/c/Users/Othman/Desktop/TP-LOG6302/log6302a_lab5/part_1/file_1.php.ast.json')

def referencesExpression (cfg : CFG, node, ref):
    
    if cfg.get_type(node) == 'BinOP':
        referencesExpression(cfg, cfg.get_op_hands(node)[0], ref)
        referencesExpression(cfg, cfg.get_op_hands(node)[1], ref)
    else : 
        ref.append(node)
    return ref

def poss_tainted_defs(cfg : CFG ) :

    taint_file = open('/mnt/c/Users/Othman/Desktop/TP-LOG6302/log6302a_lab5/part_1/file_2.php.taint.json')
    data = json.load(taint_file)
    defs = data['defs']
    refs = data['refs']
    pairs = data['pairs']
    sinks = data['sinks']
    filters = data['filters']
    safes = data['safes']
    sources = data['sources']


    nodeSet = cfg.get_node_ids()

    IN = {}
    OUT = {}
    tainted_KILL = {}
    tainted_GEN = {}

    for node in nodeSet :
        
        IN[node] = set()
        OUT[node] = set()
        tainted_GEN[node] = set()
        tainted_KILL[node] = set()

        if cfg.get_type(node) == 'BinOP' and cfg.get_image(node) == '=' :
            left, right = cfg.get_op_hands(node)
            if right in sources :
                tainted_GEN[node].add(left)
            elif right in safes :
                tainted_GEN[node] = set()
            elif right in filters :
                tainted_GEN[node] = set()
            else : 
                refExpr = referencesExpression (cfg , right, [])
                for ref in refExpr :
                    for defRef in pairs :
                        if defRef[1] == ref and defRef[0] in IN[node]:
                            tainted_GEN[node].add(left)
                            break
            
    print(tainted_GEN)

poss_tainted_defs(cfg)
                
            


[43, 45, 46]
[50, 51]
{64: set(), 65: set(), 66: set(), 67: set(), 68: set(), 30: set(), 31: set(), 32: set(), 33: set(), 34: set(), 35: set(), 36: {37}, 37: set(), 38: set(), 39: set(), 40: set(), 41: set(), 42: set(), 43: set(), 44: set(), 45: set(), 46: set(), 47: set(), 48: set(), 49: set(), 50: set(), 51: set(), 52: set(), 53: set(), 54: set(), 55: set(), 56: set(), 57: set(), 58: set(), 59: set(), 60: set(), 61: set(), 62: set(), 63: set()}
